In [1]:
import tensorflow as tf
import requests
import pandas as pd
import os
import base64
import json

In [4]:
DATA_ROOT = "dataset"

In [5]:
df = pd.read_csv(DATA_ROOT + "/fake_job_postings_clean.csv")

In [6]:
test_input_fake = df['full_description'][0]
test_input_real = df['full_description'][3]

In [7]:
def to_request(text):
    feature_spec = {
        "full_description": tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(text, "utf-8")]))
    }
    
    serializeToString = tf.train.Example(features=tf.train.Features(feature=feature_spec)).SerializeToString()
    
    result = [
        {
            "examples": {
                "b64": base64.b64encode(serializeToString).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result
    })

In [8]:
def predict(input):
    res = requests.post("http://103.190.215.94:8501/v1/models/real-or-fake-jobs-detection-model:predict", data=to_request(input)).json()
    return "Real Job" if res['predictions'][0][0] > 0.7 else "Fake Job"

In [9]:
predict(test_input_fake)

'Fake Job'

In [10]:
predict(test_input_real)

'Real Job'